<a href="https://colab.research.google.com/github/HONAJIPHAD/Topic-Modeling-On-News-Article/blob/main/Topic_Modeling_On_News_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Title : Extraction/identification of major topics & themes discussed in news articles.


##Problem Description

In this project your task is to identify major themes/topics across a collection of BBC news articles. You can use clustering algorithms such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA) etc.

##Data Description

The dataset contains a set of news articles for each major segment consisting of business, entertainment, politics, sports and technology. You need to create an aggregate dataset of all the news articles and perform topic modeling on this dataset. Verify whether these topics correspond to the different tags available.

In [7]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import accuracy_score, classification_report
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Importing the dataset
data = pd.read_csv('/content/drive/MyDrive/Capston Projects/New folder/bbc.csv')

#First Look of Data

In [11]:
data.head()

,Unnamed: 0,news,type
0,0,Disney settles disclosure charges\n \n Walt Di...,business
1,1,Asia shares defy post-quake gloom\n \n Thailan...,business
2,2,India-Pakistan peace boosts trade\n \n Calmer ...,business
3,3,Quake's economic costs emerging\n \n Asian gov...,business
4,4,Ailing EuroDisney vows turnaround\n \n EuroDis...,business


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2225 non-null   int64 
 1   news        2225 non-null   object
 2   type        2225 non-null   object
dtypes: int64(1), object(2)
memory usage: 52.3+ KB
